In [1]:
import subprocess
import re
import argparse

In [2]:
class NameException(Exception): pass
subprocess.call("pip3 install pandas",shell=True)
subprocess.call("pip3 install requests",shell= True)

0

In [3]:
import pandas as pd
import numpy as np
import requests

In [4]:
def loadIDtable(IDtable):
    SpeciesTable = pd.read_csv(IDtable,names=['GeneBnakAssemblyID','GenBankreleaseID','RefSeqAssemblyID','RefSeqreleaseID'],
    sep='\t',header=0,index_col=False)
    return SpeciesTable

In [11]:
if __name__ == "__main__":
    
    inputName = 'Mycobacterium_abscessus'
    DataOrigin = 'Ref'
    SpeciesTable = loadIDtable('./IDtable_test.txt')

    if DataOrigin == 'GB':
        TableColumn = 0
    elif DataOrigin == "Ref":
        TableColumn = 2
    else:
        raise NameException("Error: please choose database: GB/Ref!")

    SpeciesAssembly = SpeciesTable[SpeciesTable.columns[TableColumn]]
    
    count = 0
    ContextFilename = inputName + '_Context.txt'
    with open(ContextFilename, "w") as f:
        for i in SpeciesAssembly:
            try:
                TestUrl = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/'+ i[0:3] +'/'+ i[4:7] +'/'+ i[7:10] + '/'+ i[10:13] +'/'
                request = requests.get(TestUrl)
                raw_list = re.compile(r'<a.*?>(.*?)</a>').finditer(request.text.strip())

                for j in raw_list:
                    x = j.group(1)
                    if x[0:3] == 'GCF':
                        count = count + 1
                        fnafile = x[:-1] + "_genomic.fna.gz"
                        finalUrl_https = TestUrl + x + fnafile
                        finalUrl_rsync = finalUrl_https.replace("https","rsync")
                            
                        f.write(finalUrl_rsync)
                        f.write('\n')
                        #print(finalUrl_rsync)
            except TypeError as E:
                pass
    print("Finish search from NCBI:",inputName,"\tToal:",count)

Finish search from NCBI: Mycobacterium_abscessus 	Toal: 3


TypeError: 'float' object is not subscriptable